# Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

In [34]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [35]:
from crewai import Agent, Task, Crew

In [36]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [37]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [38]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [39]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [40]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [41]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [42]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [43]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [44]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [45]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [46]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2025-04-20 18:45:57,032 - 4634392064 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [47]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'BITCOIN',
    'initial_capital': '10000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Long Term',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [48]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (BITCOIN). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for BITCOIN to the Data Analyst, as they are skilled in statistical modeling and machine learning.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for BITCOIN using statistical modeling and machine learning",
    "context": "The task involves analyzing market data for BITCOIN to identify trends and predict market movements. Utilize statistical modeling and machine learning techniques to gain insights and provide alerts about significant market opportunities or threats."
}


> Entering new CrewAgentExecutor chain...
I should start by readin

- Display the final result as Markdown.

In [49]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Bitcoin Trading Strategies:

1. Introduction:
Bitcoin trading involves various strategies such as day trading, swing trading, scalping, arbitrage, buy and hold, and using deep Q-networks. Each strategy has its own set of risks and potential rewards. This report aims to analyze the risks associated with these strategies and provide mitigation recommendations for Bitcoin trading.

2. Risks Analysis:
a. Day Trading: The main risk of day trading is high volatility in Bitcoin prices, leading to potential losses if not executed properly. Mitigation: Implement strict stop-loss orders and set clear entry and exit points to minimize risks.
b. Swing Trading: Risk involves holding positions for longer periods, exposing traders to market fluctuations. Mitigation: Conduct thorough technical analysis and consider market trends before entering swing trades.
c. Scalping: Risk lies in quick market movements and execution delays. Mitigation: Use advanced trading tools for quick order execution and set tight stop-loss orders.
d. Arbitrage: Risk arises from price differentials between exchanges and delays in transactions. Mitigation: Monitor exchange rates closely and execute trades swiftly to capitalize on price differentials.
e. Buy and Hold: Risk includes long-term price fluctuations and market uncertainty. Mitigation: Diversify investments and stay informed about market trends to make informed decisions.
f. Deep Q-networks: Risk involves algorithmic errors and technical glitches. Mitigation: Regularly update and test trading algorithms to ensure accuracy and efficiency.

3. Mitigation Strategies:
- Implement risk management techniques such as diversification, stop-loss orders, and position sizing to minimize potential losses.
- Stay informed about market news and events that could impact Bitcoin prices and adjust trading strategies accordingly.
- Continuously monitor and evaluate trading strategies to adapt to changing market conditions and optimize performance.
- Collaborate with other traders and industry experts to gain insights and refine trading strategies for better risk management.

4. Conclusion:
In conclusion, trading Bitcoin involves inherent risks that can be managed through thorough analysis, strategic planning, and disciplined execution. By understanding the risks associated with different trading strategies and implementing effective mitigation strategies, traders can navigate the volatile market environment and maximize their trading potential while minimizing potential losses.